In [6]:
!pip install deep-sort-realtime

In [40]:
import cv2
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.transforms import functional as F
from deep_sort_realtime.deepsort_tracker import DeepSort
import os

In [41]:
# Preparing the Faster RCNN Detector
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
fasterRCNN = fasterrcnn_resnet50_fpn(pretrained=True)
fasterRCNN.eval()
fasterRCNN.to(device)

# This will vary model-to-model : For Faster RCNN, the labels are from MS COCO
vehicle_classes = [2, 3, 4, 6]

In [42]:
# TRACKER
deepsort = DeepSort()

In [53]:
class VehicleTracker:
    def __init__(self, detector, vehicle_classes, tracker):
        self.detector = detector
        self.vehicle_classes = vehicle_classes
        self.tracker = tracker
        
    def writeVideo(self, frames, output_file='output.mp4', fps=15):
        height, width, _ = frames[0].shape
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
        for frame in frames:
            out.write(frame)
        out.release()
    
    def getVideo(self, videofile, outputfile):
        vehicle_count = 0
        cap = cv2.VideoCapture(videofile)
        framect, framecap = 0, 200
        video_frames = []
        
        while True : 
            ret, frame = cap.read()
            framect += 1
            if (not ret) or (framect > framecap):
                break

            frame_tensor = F.to_tensor(frame).unsqueeze(0).to(device)
            with torch.no_grad():
                outputs = self.detector(frame_tensor)

            boxes = outputs[0]['boxes'].cpu().numpy()
            scores = outputs[0]['scores'].cpu().numpy()
            labels = outputs[0]['labels'].cpu().numpy()
            detections = [(box, score, label) for box, score, label in zip(boxes, scores, labels) if score > 0.8 and label in self.vehicle_classes]

            # Update tracker with detections
            tracked_objects = self.tracker.update_tracks(detections, frame = frame)

            video_frame = frame.copy()
            for obj in tracked_objects : 
                track_id = obj.track_id
                bbox = obj.to_ltrb().astype("int")
                cv2.rectangle(video_frame, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (int(track_id), 255, int(track_id)), 2)
                cv2.putText(video_frame, str(track_id), (bbox[0], bbox[1] - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2)
            video_frames.append(video_frame)

        #     for obj in tracked_objects:
        #         if obj['bbox'][1] > intersection_y_threshold:
        #             vehicle_count += 1

        #     cv2.putText(frame, f'Vehicle Count: {vehicle_count}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cap.release()
        self.writeVideo(video_frames, output_file = outputfile)

vehicleTracker = VehicleTracker(fasterRCNN, vehicle_classes, deepsort)         

In [54]:
vehicleTracker.getVideo('/kaggle/input/intersectiondata01/vid3.mp4', "output1.mp4")